# TP 3 : Branch-and-bound applied to a knapsack problem

### Initialisation (à faire une seule fois)

In [1]:
import Pkg; 
Pkg.add("GraphRecipes"); Pkg.add("Plots"); 
using GraphRecipes, Plots #only used to visualize the search tree at the end of the branch-and-bound

   Updating registry at `~/.julia/registries/General`
  Resolving package versions...
  Installed Netpbm ───────────── v1.1.0
  Installed GraphRecipes ─────── v0.5.5
  Installed ImageAxes ────────── v0.6.10
  Installed GraphPlot ────────── v0.5.2
  Installed StatsBase ────────── v0.33.21
  Installed IJulia ───────────── v1.23.3
  Installed Compose ──────────── v0.9.4
  Installed ImageMetadata ────── v0.9.8
  Installed ImageBase ────────── v0.1.5
  Installed GeometryBasics ───── v0.3.10
  Installed ImageCore ────────── v0.9.4
  Installed NetworkLayout ────── v0.2.0
  Installed ColorSchemes ─────── v3.20.0
  Installed TranscodingStreams ─ v0.9.10
  Installed MozillaCACerts_jll ─ v2022.10.11+0
  Installed StructArrays ─────── v0.4.2
  Installed Parsers ──────────── v2.4.2
  Installed Interpolations ───── v0.12.10
######################################################################### 100,0%
   Updating `~/.julia/environments/v1.4/Project.toml`
  [13f3f980] ↓ CairoMakie v0.7.2 ⇒ v0.6.4
 

### Récupération des données

In [1]:
function readKnaptxtInstance(filename)
    price=[]
    weight=[]
    KnapCap=[]
    open(filename) do f
        for i in 1:3
            tok = split(readline(f))
            if(tok[1] == "ListPrices=")
                for i in 2:(length(tok)-1)
                    push!(price,parse(Int64, tok[i]))
                end
            elseif(tok[1] == "ListWeights=")
                for i in 2:(length(tok)-1)
                    push!(weight,parse(Int64, tok[i]))
                end
            elseif(tok[1] == "Capacity=")
                push!(KnapCap, parse(Int64, tok[2]))
            else
                println("Unknown read :", tok)
            end 
        end
    end
    capacity=KnapCap[1]
    return price, weight, capacity
end

readKnaptxtInstance (generic function with 1 method)

### Tests de sondabilités TA, TO et TR basés sur le modèle linéaire

In [2]:
function TestsSondabilite_LP(model2, x, BestProfit, Bestsol)
    TA, TO, TR = false, false, false
    if(termination_status(model2) == MOI.INFEASIBLE)#Test de faisabilite
        TA=true
        println("TA")
    elseif(objective_value(model2) <= BestProfit) #Test d'optimalite
        TO=true
        println("TO")
    elseif( prod(abs.([round.(v, digits=0) for v in value.(x)]-value.(x)) .<= fill(10^-5, size(x))) 
        ) #Test de resolution
        TR=true
        println("TR")
        #if (value(benef) >= BestProfit)
        if (objective_value(model2) >= BestProfit)
            Bestsol = value.(x)
            #BestProfit=value(benef)
            BestProfit=objective_value(model2)
            println("\nNew Solution memorized ", Bestsol, " with bestprofit ", BestProfit, "\n")
        end
    else
        println("non sondable")
    end
    TA, TO, TR, Bestsol, BestProfit
end

TestsSondabilite_LP (generic function with 1 method)

## Procédure de séparation (branching) et stratégie d'exploration permettant de se placer au prochain noeud à traiter

In [3]:

function SeparerNoeud_lexicographic_depthfirst!(listobjs, listvals, n)
    # this node is non-sondable. Apply the branching criterion to separate it into two subnodes
    # and choose the child-node at the left  
    
    # lexicographic branching criterion: branch on the 1st object not yet fixed
    i, obj = 1, 0
    while((i <= n) && (obj==0))
        if(!(i in listobjs))
            obj=i
        end
        i+=1
    end
    
    println("\nbranch on object ", obj, "\n")

    # depthfirst exploration strategy: the node selected will be the most left of the child-nodes just created
    push!(listobjs,obj) #save the identity of the object selected for branching
    push!(listvals,1.0) #save the node selected, identified by the value assigned to the variable/object chosen
end


function ExplorerAutreNoeud_depthfirst!(listobjs, listvals, listnodes)
    #this node is sondable, go back to parent node then right child if possible
    
    stop=false
    #check if we are not at the root node
    if (length(listobjs)>= 1)
        #go back to parent node
        obj=pop!(listobjs)
        theval=pop!(listvals)
        tmp=pop!(listnodes)

        #go to right child if possible, otherwise go back to parent
        while( (theval==0.0) && (length(listobjs)>= 1))
            obj=pop!(listobjs)
            theval=pop!(listvals)
            tmp=pop!(listnodes)
        end
        if theval==1.0
            push!(listobjs,obj)
            push!(listvals,0.0)
        else
            println("\nFINISHED")
            stop=true
        end
    else
        #the root node was sondable
        println("\nFINISHED")
        stop=true
    end
    return stop 
end

ExplorerAutreNoeud_depthfirst! (generic function with 1 method)

###  Création de la relaxation linéaire (= modèle associé au noeud 0): <span style="color:red"> SECTION A SUPPRIMER !!!! </span>

<span style="color:red"> Cette section est à commenter/supprimer et remplacer par vos propres calculs de bornes supérieures et autres, par exemple basées sur les bornes 1 et 2 vues en cours, ou d'autres calculs de bornes de votre choix/conception validés au préalable par votre encadrant/e de TP </span>

In [4]:
Pkg.add("Clp");
Pkg.add("JuMP");
using JuMP, Clp

function MajModele_LP!(model2, x, listobjs, listvals)
    for i in 1:length(listobjs)
        set_lower_bound(x[listobjs[i]],listvals[i])
        set_upper_bound(x[listobjs[i]],listvals[i])
    end
end

function Reset_LP!(model2, x, listobjs)
    for i in 1:length(listobjs)
        set_lower_bound(x[listobjs[i]],0.0)
        set_upper_bound(x[listobjs[i]],1.0)
    end
end

function Reset_allLP!(model2, x)
    for i in 1:length(x)
        set_lower_bound(x[i],0.0)
        set_upper_bound(x[i],1.0)
    end
end

function CreationModele_LP(price, weight, capacity)
# ROOT NODE
    
    n=length(price)
    
    model2 = Model(Clp.Optimizer) # set optimizer
    set_optimizer_attribute(model2, "LogLevel", 0) #don't display anything during solve
    set_optimizer_attribute(model2, "Algorithm", 4) #LP solver chosen is simplex

    # define x variables as CONTINUOUS (recall that it is not possible to define binary variables in Clp)
    @variable(model2, 0 <= x[i in 1:n] <= 1)

    # define objective function
    @objective(model2, Max, sum(price[i]*x[i] for i in 1:n))

    # define the capacity constraint 
    @constraint(model2, sum(weight[i]*x[i] for i in 1:n) <=  capacity)

    println(model2) 

    return model2, x
end

UndefVarError: UndefVarError: Pkg not defined

### Boucle principale : résoudre une relaxation, appliquer les tests de sondabilité, identifier le prochain noeud, répéter.

In [5]:

#=
function SolveKnapInstance(filename)

    price, weight, capacity = readKnaptxtInstance(filename)

    model2, x = CreationModele_LP(price, weight, capacity)

    #create the structure to memorize the search tree for visualization at the end
    trParentnodes=Int64[] #will store orig node of arc in search tree
    trChildnodes=Int64[] #will store destination node of arc in search tree
    trNamenodes=[] #will store names of nodes in search tree

    #intermediate structure to navigate in the search tree
    listobjs=[]
    listvals=[]
    listnodes=[]

    BestProfit=-1
    Bestsol=[]

    current_node_number=0
    stop = false

    while(!stop)

        println("\nNode number ", current_node_number, ": \n---------------\n")

        #Update the graphical tree
        push!(trNamenodes,current_node_number+1) 
        if(length(trNamenodes)>=2)
            push!(trParentnodes,listnodes[end]+1) # +1 because the 1st node is "node 0"
            push!(trChildnodes, current_node_number+1) # +1 because the 1st node is "node 0"
        end
        push!(listnodes, current_node_number)

        
        #create LP of current node
        MajModele_LP!(model2, x, listobjs, listvals)
        
        println(model2)
        
        print("Solve the LP model of the current node to compute its bound: start ... ")

        status = optimize!(model2)

        println("... end"); 

        print(": Solution LP")
        if(termination_status(model2) == MOI.INFEASIBLE)#(has_values(model2))
            print(" : NOT AVAILABLE (probably infeasible or ressources limit reached)")
        else
            print(" ", objective_value(model2))
            [print("\t", name(v),"=",value(v)) for v in all_variables(model2)] 
        end
        println(" "); 


        println("\nPrevious Solution memorized ", Bestsol, " with bestprofit ", BestProfit, "\n")

        TA, TO, TR, Bestsol, BestProfit = TestsSondabilite_LP(model2, x, BestProfit, Bestsol)

        is_node_sondable = TA || TO || TR

        #Reset_LP!(model2, x, listobjs)

        if(!is_node_sondable)
            SeparerNoeud_lexicographic_depthfirst!(listobjs, listvals, length(price))
        else
            stop = ExplorerAutreNoeud_depthfirst!(listobjs, listvals, listnodes)
        end
        
        Reset_allLP!(model2, x)

        current_node_number = current_node_number + 1
    end

    println("\n******\n\nOptimal value = ", BestProfit, "\n\nOptimal x=", Bestsol)

    return BestProfit, Bestsol, trParentnodes, trChildnodes, trNamenodes

end
=#

In [16]:
function constructionC(price, weight, capacity)
    # Fonction pour construire la matrice C
    c = zeros(length(price),capacity+1)

    i = 1

    while(i<= length(price))

        j = 2

        while(j<=capacity+1)
            i_prec = max(i-1,1)
            j_moins_poids = max(1,j-weight[i])

            c_prec = c[i_prec,j]

            c_actuel_ajout = c[i_prec,j_moins_poids]+price[i]

            if (weight[i]>j-1)
                c_actuel_ajout = 0
            end

            c[i,j] = max(c_prec,c_actuel_ajout)

            j+=1

        end

        i+=1

    end

    return c

end

function analyseC(c,price,weight,capacity)
    Bestsol, BestProfit = zeros(length(price)), -1
    i, j = length(price), capacity + 1

    BestProfit = c[i,j]
    while (i > 1)
        if c[i-1,j] == c[i,j]
            Bestsol[i] = 0.0
        else
            Bestsol[i] = 1.0
            j -= weight[i]
            if j <= 0
                j = 1
            end
        end
        i-=1
    end

    if c[i,j] != 0
        Bestsol[i] = 1.0

    end 

    return Bestsol, BestProfit
end

function SolveKnapInstance(filename)

    price, weight, capacity = readKnaptxtInstance(filename)

    println("\nConstruction de la matrice C ...")
    c = constructionC(price, weight, capacity)
    println("\n... Matrice C contruite")

    #println("\nMatrice C obtenue ",c)
    
    println("\nAnalyse de la matrice C ...\n")

    Bestsol,BestProfit = analyseC(c,price, weight, capacity)
    println("\n... Matrice C analysée")
    println("\n******\n\nOptimal value = ", BestProfit, "\n\nOptimal x=", Bestsol)

    return BestProfit,Bestsol

end

SolveKnapInstance (generic function with 1 method)

### Affichage du résultat final

In [7]:
BestProfit, Bestsol = SolveKnapInstance("instancesETU/KNAPnewformat/test.opb.txt")
println("\n******\n\nOptimal value = ", BestProfit, "\n\nOptimal x=", Bestsol)



Construction de la matrice C


Analyse de la matrice C


******

Optimal value = 65.0



Optimal x=

[0.0, 1.0, 0.0, 1.0]

******

Optimal value = 65.0

Optimal x=[0.0, 1.0, 0.0, 1.0]


In [12]:
BestProfit, Bestsol = SolveKnapInstance("instancesETU/KNAPnewformat/strongly_correlated/knapPI_3_50_1000_1_-1894.opb.txt")


Construction de la matrice C


Analyse de la matrice C




******

Optimal value = 1894.0

Optimal x=[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]


In [15]:
BestProfit, Bestsol = SolveKnapInstance("instancesETU/KNAPnewformat/uncorrelated/knapPI_1_50_1000_1_-8373.opb.txt")


Construction de la matrice C


Analyse de la matrice C



(8373.0, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0])

In [14]:
BestProfit, Bestsol = SolveKnapInstance("instancesETU/KNAPnewformat/weakly_correlated/knapPI_2_50_1000_1_-1396.opb.txt")


Construction de la matrice C


Analyse de la matrice C


******

Optimal value = 1396.0

Optimal x=[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [22]:
cd("/home/romai/Enseeiht/S7/RechOp/TPs/instancesETU_TP3/instancesETU/KNAPnewformat")
listeDossier = readdir()
for dossier in listeDossier
    if isdir(dossier)
        cd(dossier)
        listeTest = readdir()
        for test in listeTest
            println("\n",test,"\n")
            try 
                _, _ = SolveKnapInstance("instancesETU/KNAPnewformat/weakly_correlated/knapPI_2_50_1000_1_-1396.opb.txt")
            catch e
                println("Error test")
            end
        end
        cd("..")
    end
end
        


knapPI_5_10000_10000_1_-1484157.opb.txt

Error test

knapPI_5_10000_10000_2_-2405704.opb.txt

Error test

knapPI_5_10000_10000_3_-3195902.opb.txt

Error test

knapPI_5_10000_10000_4_-4012574.opb.txt

Error test

knapPI_5_10000_10000_5_-4687981.opb.txt

Error test

knapPI_5_10000_1000_1_-145393.opb.txt

Error test

knapPI_5_10000_1000_2_-238597.opb.txt

Error test

knapPI_5_10000_1000_3_-320932.opb.txt

Error test

knapPI_5_10000_1000_4_-396116.opb.txt

Error test

knapPI_5_10000_1000_5_-466744.opb.txt

Error test

knapPI_5_1000_10000_1_-146390.opb.txt

Error test

knapPI_5_1000_10000_2_-243637.opb.txt

Error test

knapPI_5_1000_10000_3_-322069.opb.txt

Error test

knapPI_5_1000_10000_4_-393112.opb.txt

Error test

knapPI_5_1000_10000_5_-468478.opb.txt

Error test

knapPI_5_1000_1000_1_-14202.opb.txt

Error test

knapPI_5_1000_1000_2_-24297.opb.txt

Error test

knapPI_5_1000_1000_3_-31556.opb.txt

Error test

knapPI_5_1000_1000_4_-40866.opb.txt

Error test

knapPI_5_1000_1000_5_-46434.


knapPI_14_10000_1000_4_-634984.opb.txt

Error test

knapPI_14_10000_1000_5_-726720.opb.txt

Error test

knapPI_14_1000_1000_1_-26616.opb.txt

Error test

knapPI_14_1000_1000_2_-38335.opb.txt

Error test

knapPI_14_1000_1000_3_-54883.opb.txt

Error test

knapPI_14_1000_1000_4_-63934.opb.txt

Error test

knapPI_14_1000_1000_5_-72286.opb.txt

Error test

knapPI_14_100_1000_1_-3697.opb.txt

Error test

knapPI_14_100_1000_2_-3482.opb.txt

Error test

knapPI_14_100_1000_3_-5765.opb.txt

Error test

knapPI_14_100_1000_4_-6730.opb.txt

Error test

knapPI_14_100_1000_5_-6611.opb.txt

Error test

knapPI_14_2000_1000_1_-54180.opb.txt

Error test

knapPI_14_2000_1000_2_-79605.opb.txt

Error test

knapPI_14_2000_1000_3_-108409.opb.txt

Error test

knapPI_14_2000_1000_4_-128348.opb.txt

Error test

knapPI_14_2000_1000_5_-145151.opb.txt

Error test

knapPI_14_200_1000_1_-5397.opb.txt

Error test

knapPI_14_200_1000_2_-6898.opb.txt

Error test

knapPI_14_200_1000_3_-11972.opb.txt

Error test

knapPI_



Error test

knapPI_13_100_1000_1_-1989.opb.txt

Error test

knapPI_13_100_1000_2_-2806.opb.txt

Error test

knapPI_13_100_1000_3_-1760.opb.txt

Error test

knapPI_13_100_1000_4_-1734.opb.txt

Error test

knapPI_13_100_1000_5_-2601.opb.txt

Error test

knapPI_13_2000_1000_1_-13026.opb.txt

Error test

knapPI_13_2000_1000_2_-36051.opb.txt

Error test

knapPI_13_2000_1000_3_-33075.opb.txt

Error test

knapPI_13_2000_1000_4_-37058.opb.txt

Error test

knapPI_13_2000_1000_5_-50848.opb.txt

Error test

knapPI_13_200_1000_1_-1989.opb.txt

Error test

knapPI_13_200_1000_2_-3416.opb.txt

Error test

knapPI_13_200_1000_3_-3485.opb.txt

Error test

knapPI_13_200_1000_4_-3680.opb.txt

Error test

knapPI_13_200_1000_5_-5202.opb.txt

Error test

knapPI_13_20_1000_1_-1716.opb.txt

Error test

knapPI_13_20_1000_2_-2501.opb.txt

Error test

knapPI_13_20_1000_3_-1200.opb.txt

Error test

knapPI_13_20_1000_4_-907.opb.txt

Error test

knapPI_13_20_1000_5_-1326.opb.txt

Error test

knapPI_13_5000_1000_1_



Error test

knapPI_11_100_1000_4_-2580.opb.txt

Error test

knapPI_11_100_1000_5_-2392.opb.txt

Error test

knapPI_11_2000_1000_1_-14076.opb.txt

Error test

knapPI_11_2000_1000_2_-66310.opb.txt

Error test

knapPI_11_2000_1000_3_-45360.opb.txt

Error test

knapPI_11_2000_1000_4_-53660.opb.txt

Error test

knapPI_11_2000_1000_5_-47748.opb.txt

Error test

knapPI_11_200_1000_1_-1428.opb.txt

Error test

knapPI_11_200_1000_2_-6460.opb.txt

Error test

knapPI_11_200_1000_3_-4500.opb.txt

Error test

knapPI_11_200_1000_4_-5180.opb.txt

Error test

knapPI_11_200_1000_5_-4723.opb.txt

Error test

knapPI_11_20_1000_1_-1428.opb.txt

Error test

knapPI_11_20_1000_2_-3990.opb.txt

Error test

knapPI_11_20_1000_3_-1800.opb.txt

Error test

knapPI_11_20_1000_4_-1440.opb.txt

Error test

knapPI_11_20_1000_5_-736.opb.txt

Error test

knapPI_11_5000_1000_1_-35292.opb.txt

Error test

knapPI_11_5000_1000_2_-166440.opb.txt

Error test

knapPI_11_5000_1000_3_-114030.opb.txt

Error test

knapPI_11_5000

Error test

knapPI_12_100_1000_1_-970.opb.txt

Error test

knapPI_12_100_1000_2_-940.opb.txt

Error test

knapPI_12_100_1000_3_-1700.opb.txt

Error test

knapPI_12_100_1000_4_-2739.opb.txt

Error test

knapPI_12_100_1000_5_-2871.opb.txt

Error test

knapPI_12_2000_1000_1_-9118.opb.txt

Error test

knapPI_12_2000_1000_2_-14968.opb.txt

Error test

knapPI_12_2000_1000_3_-34272.opb.txt

Error test

knapPI_12_2000_1000_4_-56108.opb.txt

Error test

knapPI_12_2000_1000_5_-57519.opb.txt

Error test

knapPI_12_200_1000_1_-970.opb.txt

Error test

knapPI_12_200_1000_2_-1452.opb.txt

Error test

knapPI_12_200_1000_3_-3400.opb.txt

Error test

knapPI_12_200_1000_4_-5478.opb.txt

Error test

knapPI_12_200_1000_5_-5742.opb.txt

Error test

knapPI_12_20_1000_1_-970.opb.txt

Error test

knapPI_12_20_1000_2_-940.opb.txt

Error test

knapPI_12_20_1000_3_-1360.opb.txt

Error test

knapPI_12_20_1000_4_-1494.opb.txt

Error test

knapPI_12_20_1000_5_-990.opb.txt

Error test

knapPI_12_5000_1000_1_-22847.o